In [1]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade pip setuptools


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy==1.7.3


Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 1.11.0
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11; 1.9.0 Requires-Python >=3.8,<3.12; 1.9.0rc1 Requires-Python >=3.8,<3.12; 1.9.0rc2 Requires-Python >=3.8,<3.12; 1.9.0rc3 Requires-Python >=3.8,<3.12; 1.9.1 Requires-Python >=3.8,<3.12
ERROR: Could not find a version that satisfies the requirement scipy==1.7.3 (from versions: 0.8.0, 0.9.

In [4]:
pip install -r requirements.txt


  Using cached transformers-3.4.0-py3-none-any.whl.metadata (31 kB)
  Using cached pytest-5.3.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached scikit-learn-0.22.1.tar.gz (6.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tokenizers-0.9.2.tar.gz (170 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Us

  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      <string>:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      Partial import of sklearn during the build process.
      Traceback (most recent call last):
        File "<string>", line 195, in check_package_status
        File "c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\importlib\__init__.py", line 90, in import_module
          return _bootstrap._gcd_import(name[level:], package, level)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
        File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
        File "<frozen importlib._bootstrap>", line 1324, in _find_and_load_unlocked
      ModuleNotFoundError: No module named 'n

In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip setup.py install 


Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "setup.py"



In [7]:
pip setup.py develop

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "setup.py"



In [8]:

import json
import urllib
import opennre

In [9]:
import spacy
from typing import List
from spacy.tokens import Doc, Span
from fastcoref import FCoref
#from fastcoref import LingMessCoref
# from flask import Flask, render_template, request, redirect, session, jsonify

nlp = spacy.load('en_core_web_sm')
model = FCoref()
#model = LingMessCoref(device='cuda:0')
# app = Flask(__name__)

def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved

def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]

def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

def get_fast_cluster_spans(doc, clusters):
    fast_clusters = []
    for cluster in clusters:
        new_group = []
        for tuple in cluster:
            print(type(tuple), tuple)
            (start, end) = tuple
            print("start, end", start, end)
            span = doc.char_span(start, end)
            print('span', span.start, span.end)
            new_group.append([span.start, span.end-1])
        fast_clusters.append(new_group)
    return fast_clusters

def get_fastcoref_clusters(doc, text):
    preds = model.predict(texts=[text])
    fast_clusters = preds[0].get_clusters(as_strings=False)
    fast_cluster_spans = get_fast_cluster_spans(doc, fast_clusters)
    return fast_cluster_spans

# @app.route('/coreference', methods=['POST'])
def coreference(text):
    #print(request.form["test"])
    # content = request.get_json()
    # print('content', content)
    # text = content["text"]
   
    doc = nlp(text)
    #clusters = get_allennlp_clusters(text)
    clusters = get_fastcoref_clusters(doc, text)
    coref_text = improved_replace_corefs(doc, clusters)
    print('coref_text', coref_text)
    return coref_text


2024-04-14 00:26:02,688 - datasets - INFO - PyTorch version 2.2.2+cu118 available.
2024-04-14 00:26:05,754 - fastcoref.modeling - INFO - missing_keys: []
2024-04-14 00:26:05,755 - fastcoref.modeling - INFO - unexpected_keys: []
2024-04-14 00:26:05,756 - fastcoref.modeling - INFO - mismatched_keys: []
2024-04-14 00:26:05,756 - fastcoref.modeling - INFO - error_msgs: []
2024-04-14 00:26:05,759 - fastcoref.modeling - INFO - Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [10]:
import json
import urllib
from string import punctuation
import nltk

# Rest of your code...


ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
            elif any([el['enLabel'] in ["geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [11]:

from flask import Flask, request


In [33]:
from opennre.pretrain import get_model
import itertools
import nltk
nltk.download('punkt')
relation_model = get_model('wiki80_cnn_softmax')
ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "human settlement", "geographic entity", "territorial entity type", "organization"]

def strip_punctuation(s):
    """Removes all punctuation from a string"""
    return ''.join(c for c in s if c not in punctuation)


def deduplicate_dict(d):
    return [dict(y) for y in set(tuple(x.items()) for x in d)]


def final(text):
   
        relation_threshold=0.9
        entities_threshold=0.8
        coref =  True
        if not text:
            return 'Missing text parameter'

        try:
            relation_threshold = float(relation_threshold)
            entities_threshold = float(entities_threshold)
        except ValueError:
            return 'Invalid value for relation or entity threshold parameter'

        if coref:
            text = coreference(text)

        print(text)

        relations_list = list()
        entities_list = list()

        for sentence in nltk.sent_tokenize(text):
            sentence = strip_punctuation(sentence)
            entities = wikifier(sentence, threshold=entities_threshold)
            entities_list.extend(
                [{'title': el['title'], 'wikiId': el['wikiId'], 'label': el['label']} for el in entities])
            # Iterate over every permutation pair of entities
            for permutation in itertools.permutations(entities, 2):
                for source in permutation[0]['characters']:
                    for target in permutation[1]['characters']:
                        # Relationship extraction with OpenNRE
                        data = relation_model.infer(
                            {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
                        if data[1] > relation_threshold:
                            relations_list.append(
                                {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})
        result = {'entities': deduplicate_dict(entities_list), 'relations': deduplicate_dict(relations_list)}

        print("Entities:")
        for entity in entities_list:
            print(f"{entity['title']}: {entity['label']}")

        # Print relations for current sentence
        print("\nRelations:")
        for relation in relations_list:
            print(f"{relation['source']} -- {relation['type']} --> {relation['target']}")
    

text= "Akbar (Abu'l-Fath Jalal ud-b din Muhammad Akbar, 25 October 1542 – 27 October 1605), also known as Akbar the Great was the 3rd Mughal Emperor.[1] He was born in Lahore (now Pakistan). He was the son of 2nd Mughal Emperor Humayun. Akbar became the de jure king in 1556 at the age of 13 when his father died. Akbar was too young to rule, so Bairam Khan was appointed as Akbar regent and chief army commander. Soon after coming to power Akbar defeated Himu, the general of the Afghan forces, in the Second Battle of Panipat. After a few years, he ended the regency of Bairam Khan and took charge of the kingdom. He initially offered friendship to the Rajputs. However, he had to fight against some Rajputs who opposed him. In 1576 he defeated Maharana Pratap of Mewar in the Battle of Haldighati. Akbar wars made the Mughal empire more than twice as big as it had been before, covering most of the Indian subcontinent except the south (excluding the Deccan Plateau)"
# ctext=coreference(text)
final(text)


    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-04-14 01:25:18,789 - root - INFO - Initializing word embedding with word2vec.
2024-04-14 01:25:18,924 - fastcoref.modeling - INFO - Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

2024-04-14 01:25:19,027 - fastcoref.modeling - INFO - ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'tuple'> (146, 148)
start, end 146 148
span 34 35
<class 'tuple'> (184, 186)
start, end 184 186
span 44 45
<class 'tuple'> (230, 235)
start, end 230 235
span 54 55
<class 'tuple'> (290, 293)
start, end 290 293
span 68 69
<class 'tuple'> (307, 312)
start, end 307 312
span 72 73
<class 'tuple'> (368, 373)
start, end 368 373
span 85 86
<class 'tuple'> (434, 439)
start, end 434 439
span 97 98
<class 'tuple'> (541, 543)
start, end 541 543
span 120 121
<class 'tuple'> (609, 611)
start, end 609 611
span 134 135
<class 'tuple'> (666, 668)
start, end 666 668
span 144 145
<class 'tuple'> (715, 718)
start, end 715 718
span 153 154
<class 'tuple'> (728, 730)
start, end 728 730
span 157 158
<class 'tuple'> (794, 799)
start, end 794 799
span 169 170
<class 'tuple'> (202, 228)
start, end 202 228
span 49 53
<class 'tuple'> (290, 300)
start, end 290 300
span 68 70
<class 'tuple'> (339, 350)
start, end 339 350
span 80 82
<class 'tuple'> (565, 576)
start, end 565 576
span 125 127
<class 'tuple'> (

In [ ]:
text=coreference()
results=wikifier(text)
# First get all the entities in the sentence
entities = wikifier(sentence, threshold=entities_threshold)
# Iterate over every permutation pair of entities
for permutation in itertools.permutations(entities, 2):
    for source in permutation[0]['characters']:
        for target in permutation[1]['characters']:
            # Relationship extraction with OpenNRE
            data = relation_model.infer(
                {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
            if data[1] > relation_threshold:
                relations_list.append(
                    {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})